In [4]:
import requests
from requests import Session
import pandas as pd

# Public API

In [8]:
class MarketData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        self.session = Session()

    # returns all tickers
    def getAllTickers(self):
        url = self.apiurl + "/api/v1/market/allTickers"
        r = self.session.get(url)
        data = r.json()["data"]["ticker"]
        df = pd.DataFrame.from_dict(data, orient="columns")
        return df.loc[
            :,
            [
                "symbolName",
                "buy",
                "sell",
                "changeRate",
                "changePrice",
                "high",
                "low",
                "vol",
                "last",
                "averagePrice",
            ],
        ]

    # returns a single trading pair ex. 'BTC-USDT'
    def getTicker(self, symbol):
        url = self.apiurl + "/api/v1/market/orderbook/level1"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df
    
    def get24hrStats(self, symbol):
        url = self.apiurl + "/api/v1/market/stats"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df

    # returns the order book for a trading pair
    def getPartOrderBook(self, symbol, amount):  # does not required API KEY
        # amount can be 20 pieces of data or 100 pieces of data
        url = self.apiurl + "/api/v1/market/orderbook/level2_" + amount
        parameters = {"symbol": symbol, "amount": amount}
        r = self.session.get(url, params=parameters)
        bids = r.json()["data"]["bids"]
        asks = r.json()["data"]["asks"]
        dfBids = pd.DataFrame.from_dict(bids, orient="columns")
        dfAsks = pd.DataFrame.from_dict(asks, orient="columns")
        dfAsks.columns = ["", "Asks"]
        dfBids.columns = ["", "Bids"]
        return dfBids, dfAsks

    # returns the fiat price of a symbol
    def getFiatPrice(self, symbol):
        url = self.apiurl + "/api/v1/prices"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"][symbol]
        return data
    
    def getAllFutureContracts(self):
        url = 'https://api-futures.kucoin.com' + '/api/v1/contracts/active'
        r = self.session.get(url)
        data = r.json()['data']
        return data

In [6]:
kucoinAPI = MarketData()

In [4]:
kucoinAPI.getAllTickers()

,symbolName,buy,sell,changeRate,changePrice,high,low,vol,last,averagePrice
0,NKN-USDT,0.0943,0.0946,0.0021,0.0002,0.0961,0.0869,227146.45136414,0.0941,0.09249521
1,LOOM-BTC,0.00000186,0.00000187,-0.0106,-0.00000002,0.00000192,0.00000186,38642.91996302,0.00000186,0.00000194
2,JUP-ETH,0.00000241,0.00000243,0.2538,0.00000049,0.00000251,0.00000183,7507287.10250946,0.00000242,0.00000228
3,CUSD-USDT,0.9965,0.9977,0.0005,0.0005,0.9979,0.9934,140264.84295499,0.9966,0.99690928
4,LTC3L-USDT,0.007723,0.007797,0.0647,0.000471,0.007857,0.006007,8929199.9082102,0.007741,0.00696715
...,...,...,...,...,...,...,...,...,...,...
1248,FRR-USDT,0.0142,0.0143,-0.1172,-0.0019,0.0164,0.0131,16281400.41337091,0.0143,0.01520292
1249,DMTR-USDT,0.0642,0.0647,0.0682,0.0041,0.085,0.0537,4196691.19259845,0.0642,0.05906078
1250,GMB-BTC,0.0000000141,0.0000000145,-0.0068,-0.0000000001,0.0000000227,0.0000000141,57583380.0841527,0.0000000145,0.00000001
1251,TRIBE-USDT,0.2315,0.232,0.0157,0.0036,0.24,0.2258,7231.21976586,0.2317,0.23240103


In [5]:
kucoinAPI.getTicker('ETH-USDT')

,ETH-USDT
time,1654634571848
sequence,1628991764332
price,1848.68
size,0.0011032
bestBid,1848.67
bestBidSize,10.0488783
bestAsk,1848.68
bestAskSize,1.90257456


In [6]:
kucoinAPI.getPartOrderBook('ETH-USDT', '20')

(                   Bids
 0   1847.78  14.7000165
 1   1847.66   0.1702367
 2   1847.65  16.9888542
 3   1847.64       2.495
 4   1847.62   2.7312173
 5    1847.5       0.381
 6   1847.47   0.0042671
 7   1847.46       3.237
 8   1847.43      7.5067
 9   1847.42   0.0023534
 10   1847.4      0.0052
 11  1847.39   0.1254869
 12  1847.36        12.5
 13   1847.3   8.4949678
 14  1847.25   0.4956696
 15  1847.24      0.0003
 16  1847.23   7.9120275
 17  1847.22   3.6074586
 18  1847.19       1.978
 19  1847.18   0.4589514,
                    Asks
 0   1847.79      0.0018
 1   1847.92   0.0702367
 2   1847.93      7.5067
 3   1847.98      0.0043
 4   1847.99      0.0091
 5      1848      1.7167
 6   1848.01        2.35
 7   1848.03   3.4237882
 8    1848.1       0.002
 9   1848.15       0.008
 10  1848.17    0.373572
 11  1848.27       1.631
 12  1848.28   2.8306349
 13  1848.29         0.1
 14  1848.32  12.8600967
 15  1848.33    1.700102
 16  1848.35   7.5284191
 17   1848.4   1.9751834

In [7]:
kucoinAPI.getFiatPrice('ICP')

'7.09361002'

In [7]:
kucoinAPI.get24hrStats('ICP-USDT')

NameError: name 'kucoinAPI' is not defined

In [9]:
kucoinAPI.getAllFutureContracts()

{'code': '200000',
 'data': [{'symbol': 'XBTUSDTM',
   'rootSymbol': 'USDT',
   'type': 'FFWCSX',
   'firstOpenDate': 1585555200000,
   'expireDate': None,
   'settleDate': None,
   'baseCurrency': 'XBT',
   'quoteCurrency': 'USDT',
   'settleCurrency': 'USDT',
   'maxOrderQty': 1000000,
   'maxPrice': 1000000.0,
   'lotSize': 1,
   'tickSize': 1.0,
   'indexPriceTickSize': 0.01,
   'multiplier': 0.001,
   'initialMargin': 0.01,
   'maintainMargin': 0.005,
   'maxRiskLimit': 2000000,
   'minRiskLimit': 2000000,
   'riskStep': 1000000,
   'makerFeeRate': 0.0002,
   'takerFeeRate': 0.0006,
   'takerFixFee': 0.0,
   'makerFixFee': 0.0,
   'settlementFee': None,
   'isDeleverage': True,
   'isQuanto': True,
   'isInverse': False,
   'markMethod': 'FairPrice',
   'fairMethod': 'FundingRate',
   'fundingBaseSymbol': '.XBTINT8H',
   'fundingQuoteSymbol': '.USDTINT8H',
   'fundingRateSymbol': '.XBTUSDTMFPI8H',
   'indexSymbol': '.KXBTUSDT',
   'settlementSymbol': '',
   'status': 'Open',
   'f